In [1]:
import sys
import os
os.chdir("..")
import pandas as pd

In [2]:
train = pd.read_csv('data/feature/X_train.csv')

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
missing_values = train.isnull().sum()
print(missing_values[missing_values > 0])

Series([], dtype: int64)


In [5]:
train.shape

(18429, 149)

In [6]:
val = pd.read_csv('data/feature/X_val.csv')
test = pd.read_csv('data/feature/X_test.csv')

In [7]:
val.shape

(3270, 149)

In [8]:
test.shape

(3262, 149)

In [9]:
train.columns

Index(['id', 'league_id', 'stage', 'home_team_api_id', 'away_team_api_id',
       'B365H', 'B365D', 'B365A', 'BWH', 'BWD',
       ...
       'away_avg_pace_player_attr_missing',
       'away_avg_passing_skill_player_attr_missing',
       'away_avg_dribbling_skill_player_attr_missing',
       'away_avg_shooting_skill_player_attr_missing',
       'away_avg_finishing_player_attr_missing',
       'away_avg_physical_player_attr_missing',
       'away_avg_defensive_skill_player_attr_missing',
       'away_avg_crossing_player_attr_missing',
       'away_avg_heading_accuracy_player_attr_missing',
       'away_avg_penalties_player_attr_missing'],
      dtype='object', length=149)